<a href="https://colab.research.google.com/github/raakanaka/Learn-Machine-Learning/blob/main/BlogPro's_Keyword_Clustering_Script_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***BlogPro's Keyword Clustering Script v1.0*** 🚀

Github - https://github.com/iamBlogPro/KeywordClustering

*For any queries email hello@wordsigma.com*


# **Step 1 - Upload your CSV file with the keywords**


In [3]:
# Say "BlogPro is the best" twice before running the script for Good Luck.

# Now let's import all the libraries required for this script to run

from google.colab import files
import os

# Clear any existing files
!rm -f keywords.csv

# Upload new file
uploaded = files.upload()

# Get the name of the uploaded file
uploaded_file_name = next(iter(uploaded))

# Check if the file uploaded correctly and is a CSV
if not uploaded_file_name.lower().endswith('.csv'):
    raise ValueError("Please upload a CSV file.")

# Rename the uploaded file
os.rename(uploaded_file_name, 'keywords.csv')

print("Uploaded file renamed to 'keywords.csv'")


Saving keyword.csv to keyword.csv
Uploaded file renamed to 'keywords.csv'


# **Step 2 - Let's Create the *Configuration File* for the script**

---

Alright, so edit the information below - if you need to.

---



In [4]:
# Say "BlogPro is the best" twice before running the script for Good Luck.

# Now let's import all the libraries required for this script to run

import json

config = {
    "file_path": "keywords.csv",
    "output_excel_file": "clustered_keywords.xlsx",
    "outliers_excel_file": "outliers.xlsx",
    "min_cluster_size": 5,
    "language": "english",
    "calculate_probabilities": False
}

# Clear any existing files
!rm -f config.json

# Here you write the config to a json file, or you can just use the dictionary directly
with open('config.json', 'w') as config_file:
    json.dump(config, config_file)

print("Config file created.")


Config file created.


# **Step 3 - Importing all the required Python Libraries**



In [ ]:
# Say "BlogPro is the best" twice before running the script for Good Luck.

# Now let's import all the libraries required for this script to run

!pip install pandas bertopic openpyxl tqdm seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.6 MB/s eta 0:00:00


# **Step 4 - Importing and Preprocessing**

This step is automated. In this step, we're importing all of the necessary python libraries we installed above,preprocessing the keywords and getting ready for the main function

In [ ]:
import pandas as pd
from bertopic import BERTopic
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter
import logging
from logging.handlers import RotatingFileHandler
import json
from tqdm.auto import tqdm
import datetime
import seaborn as sns

# Set up logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler()
file_handler = RotatingFileHandler('clustering_with_bertopic.log', maxBytes=10485760, backupCount=3)
logging_format = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(logging_format)
file_handler.setFormatter(logging_format)
logger.addHandler(console_handler)
logger.addHandler(file_handler)

# Load configuration
def load_config(config_path='config.json'):
    with open(config_path) as config_file:
        config = json.load(config_file)
    return config

# Load the dataset
def load_data(file_path):
    df = pd.read_csv(file_path, header=None, names=['keywords'])
    return df['keywords'].tolist()

# Preprocessing the keywords
def preprocess_keywords(keywords):
    tqdm.pandas(desc="Preprocessing Keywords")
    return keywords.progress_apply(lambda x: x.lower().strip())

# **Step 5 - Perform Topic Modelling**

Once again you have to do nothing - the script fill automatically do everything and get your keywords going.

In [ ]:
# Perform topic modeling with BERTopic
def perform_topic_modeling(keywords, config):
    model = BERTopic(min_topic_size=config.get('min_cluster_size', 5),
                     language=config.get('language', 'english'),
                     calculate_probabilities=config.get('calculate_probabilities', False),
                     verbose=True)
    topics, _ = model.fit_transform(keywords)
    return topics, model

# Save results to Excel, separated by identified topics and outliers
def save_to_excel(df, file_path, outliers_file_path):
    # Split the DataFrame into topics and outliers
    df_topics = df[df['group'] != -1].sort_values('group')
    df_outliers = df[df['group'] == -1]

    # Save the topics to Excel, with grouping and coloring
    wb_topics = Workbook()
    ws_topics = wb_topics.active
    ws_topics.append(["Keyword", "Group"])  # Add column headers

    # Apply color to each cell based on the topic with a maximum of 20 colors
    max_colors = 20
    colors = sns.color_palette("hsv", max_colors).as_hex()

    grouped = df_topics.groupby('group')
    row_index = 2
    for group, data in grouped:
        topic_color = colors[group % max_colors]
        fill = PatternFill(start_color=topic_color[1:], end_color=topic_color[1:], fill_type='solid')
        for _, row in data.iterrows():
            ws_topics.append([row['keyword'], row['group']])
            cell = ws_topics.cell(row=row_index, column=2)
            cell.fill = fill
            row_index += 1

    # Adjust the column widths
    for column_cells in ws_topics.columns:
        length = max(len(str(cell.value)) for cell in column_cells)
        ws_topics.column_dimensions[get_column_letter(column_cells[0].column)].width = length

    wb_topics.save(file_path)

    # Save the outliers to a separate Excel file
    wb_outliers = Workbook()
    ws_outliers = wb_outliers.active
    ws_outliers.append(["Keyword"])  # Add column header for outliers
    for _, row in df_outliers.iterrows():
        ws_outliers.append([row['keyword']])

    wb_outliers.save(outliers_file_path)

# Main function to run topic modeling
def main():
    config = load_config()

    try:
        # Load data
        keywords = load_data(config['file_path'])
        preprocessed_keywords = preprocess_keywords(pd.Series(keywords))

        # Perform topic modeling
        labels, topic_model = perform_topic_modeling(preprocessed_keywords, config)

        # Save the clustered data
        df_clustered = pd.DataFrame({'keyword': preprocessed_keywords, 'group': labels})
        save_to_excel(df_clustered, config['output_excel_file'], config['outliers_excel_file'])

        logger.info("Topic modeling completed and results saved.")

    except Exception as e:
        logger.exception("An error occurred during topic modeling.")
        raise

if __name__ == "__main__":
    main()

# **Final Step - Save Keywords Files**

The script will create two files clustered_keywords.xlsx and outliers.xlsx

Download and Enjoy! 🚀

In [ ]:
from google.colab import files
import zipfile

# Function to zip files - because Google Chrome will have issues downoading multiple files.
def zip_files(files, zip_name):
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for file in files:
            zipf.write(file, arcname=os.path.basename(file))
    return zip_name

excel_files = ['clustered_keywords.xlsx', 'outliers.xlsx']
zip_filename = 'clustered_results.zip'
zip_files(excel_files, zip_filename)

files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>